# WandB results

In [1]:
import wandb
import numpy as np
import torch
import pandas as pd

In [2]:
def flatten_log(log):
    out = {}
    for k, v in log.items():
        if isinstance(v, dict):
            for k2, v2 in v.items():
                out[f"{k}.{k2}"] = v2
        elif isinstance(v, list):
            if len(v) == 1:
                out[k] = v[0]
        else:
            out[k] = v
    return out


def handle_v(v):
    if isinstance(v, list):
        return handle_v(v[0])
    if isinstance(v, np.float32):
        return v.item()
    if isinstance(v, torch.Tensor):
        # Handle torch tensors
        if v.numel() == 1:
            return v.item()
        else:
            # Return the numpy array if it has more than one element
            return v.cpu().numpy()
    return v

In [3]:
def get_wandb_logs(api, run_path):
    # Fetch the run
    run = api.run(run_path)
    scan_history = run.scan_history()  # Fetch the scan history

    # Flatten the logs
    flattened_logs = [flatten_log(log) for log in scan_history]
    all_data = pd.DataFrame(flattened_logs)
    return all_data

In [4]:
def compute_deltas(data, up_to_step=15, max_or_min="max"):
    data = data[
        [
            "step",
            "pop_best_eval",
            "median_eval",
            "mean_eval",
            "prompt",
            "running_time",
            "memory",
            "best_img.path"
        ]
    ]

    step_0 = data[data["step"] == 0]
    all_other_steps = data[data["step"] != 0]

    baseline_pop_best = step_0.groupby("prompt")["pop_best_eval"].agg(["mean"])
    pop_bests = (
        all_other_steps.loc[all_other_steps["step"] <= up_to_step]
        .groupby("prompt")[["pop_best_eval", "mean_eval", "median_eval"]]
        .agg([max_or_min])
    )

    pop_bests.columns = pop_bests.columns.map("_".join)

    joined_data = baseline_pop_best.join(pop_bests)
    joined_data["delta_max"] = joined_data[f"pop_best_eval_{max_or_min}"] - joined_data["mean"]
    joined_data["delta_mean"] = joined_data[f"mean_eval_{max_or_min}"] - joined_data["mean"]
    joined_data["delta_median"] = joined_data[f"median_eval_{max_or_min}"] - joined_data["mean"]

    # joined_data = joined_data.rename(columns={"mean": "baseline", "pop_best_eval_max": "pop_best"})
    joined_data = joined_data.reset_index()
    joined_data = joined_data.rename(
        columns={
            "mean": "baseline",
            f"pop_best_eval_{max_or_min}": "pop_best",
            f"mean_eval_{max_or_min}": "mean",
            f"median_eval_{max_or_min}": "median",
            "delta_max": "improvement_max",
            "delta_mean": "improvement_mean",
            "delta_median": "improvement_median",
        }
    )
    return joined_data

In [22]:
def compute_deltas_dnos(data, up_to_step=15, max_or_min="max"):
    data = data[["Step", "Prompt", "Reward"]]

    step_0 = data[data["Step"] == 0]
    all_other_steps = data[data["Step"] != 0]

    baseline_pop_best = step_0.groupby("Prompt")["Reward"].agg(["mean"])
    pop_bests = (
        all_other_steps.loc[all_other_steps["Step"] <= up_to_step]
        .groupby("Prompt")["Reward"]
        .agg([max_or_min])
    )
    joined_data = pop_bests.join(baseline_pop_best)
    joined_data["delta"] = joined_data[max_or_min] - joined_data["mean"]
    joined_data = joined_data.rename(columns={"mean": "baseline", f"{max_or_min}": "pop_best"})
    joined_data = joined_data.reset_index()
    joined_data = joined_data.rename(columns={"mean": "baseline", f"{max_or_min}": "pop_best", "delta": "improvement", "Prompt": "prompt"})
    return joined_data

In [6]:
def add_fields(data, fields):
    for col, val in fields.items():
        data[col] = val
    return data

In [7]:
# Initialize WandB
api = wandb.Api(timeout=60)

## SD Random

### Img Reward

In [16]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/skl06y3a")
compute_deltas(data, up_to_step=15)

,prompt,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
0,"35mm macro shot a kitten licking a baby duck, ...",1.436298,1.988246,1.319626,1.698473,0.551948,-0.116671,0.262175
1,A 1960s poster warning against climate change.,-0.485434,1.170981,0.509485,0.545399,1.656416,0.994919,1.030833
2,A 1960s yearbook photo with animals dressed as...,-1.345676,0.601206,-0.862613,-0.845156,1.946882,0.483063,0.500521
3,"A baby fennec sneezing onto a strawberry, deta...",0.750294,1.762015,0.786108,1.003516,1.011721,0.035814,0.253222
4,A banana on the left of an apple.,0.946351,1.449116,0.352018,0.640611,0.502765,-0.594333,-0.305740
...,...,...,...,...,...,...,...,...
195,Two cars on the street.,-0.021779,1.532862,0.871082,0.983268,1.554641,0.892861,1.005047
196,Two cats and one dog sitting on the grass.,1.133387,1.738019,0.579215,0.693330,0.604632,-0.554172,-0.440057
197,Two cats and three dogs sitting on the grass.,0.390415,1.813376,1.163659,1.541516,1.422960,0.773244,1.151100
198,Two cats and two dogs sitting on the grass.,0.105738,1.778616,0.842821,0.910954,1.672878,0.737083,0.805216


### HPSv2

In [19]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/bzquoitz")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.275281,0.301366,0.280917,0.282038,0.026085,0.005636,0.006757
std,0.017328,0.016452,0.015446,0.015873,0.011435,0.009267,0.009438
min,0.227539,0.258057,0.238144,0.237793,0.004639,-0.018585,-0.012329
25%,0.263367,0.289673,0.269621,0.270081,0.018799,-0.001179,-0.000732
50%,0.275024,0.303711,0.282387,0.284485,0.025635,0.006172,0.007141
75%,0.288391,0.313965,0.292622,0.294067,0.031799,0.011074,0.012085
max,0.330566,0.343750,0.314606,0.318237,0.071045,0.039276,0.043335


### CLIP

In [20]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/bfzpae40")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,31.830313,37.139063,32.623672,32.866641,5.308750,0.793359,1.036328
std,4.121937,3.664970,3.381918,3.502261,2.885185,2.193773,2.268433
min,18.593750,29.093750,21.062500,20.562500,-0.281250,-3.750000,-3.953125
25%,29.410156,34.515625,30.917969,30.906250,3.367188,-0.664062,-0.296875
50%,31.976562,37.015625,32.593750,32.812500,4.718750,0.562500,0.812500
75%,34.031250,39.664062,35.031250,35.265625,6.894531,2.062500,2.140625
max,44.468750,49.562500,41.156250,41.750000,16.328125,9.750000,10.140625


### JPEG

In [25]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/miy93gst")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,116.782780,66.915170,108.156964,105.428983,-49.867610,-8.625817,-11.353798
std,29.854665,17.825191,20.080439,20.664202,26.241667,21.975544,21.877609
min,58.554001,26.548000,64.807999,60.786499,-135.653999,-81.400375,-88.889999
25%,96.636997,55.078500,93.763767,91.465878,-65.636998,-22.977505,-24.807749
50%,114.785004,64.230000,105.787186,103.538750,-46.273998,-6.747185,-9.158749
75%,132.409252,76.063498,118.883076,116.395250,-30.921998,5.859894,3.239996
max,212.755997,136.587006,203.392944,203.583008,4.314003,43.750999,41.611496


## SD Zero-order Img Reward

### Img Reward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/4ghfz8nm")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.035915,1.534577,0.945013,1.082029,1.498662,0.909097,1.046114
std,1.058178,0.465393,0.678645,0.710983,1.007375,0.878178,0.961096
min,-2.229451,-0.129912,-1.331369,-1.479679,0.084965,-0.575460,-0.423726
25%,-0.672078,1.361493,0.469055,0.640840,0.658142,0.233622,0.300892
50%,0.137491,1.722724,1.051813,1.303073,1.265968,0.685107,0.747256
75%,0.847085,1.866370,1.502448,1.667772,2.111342,1.334105,1.521701
max,1.876520,2.006367,1.966297,1.989202,4.007376,3.532083,3.874983


### HPSv2

In [75]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/rwijixwm")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.277531,0.303896,0.289102,0.290278,0.026366,0.011572,0.012748
std,0.018112,0.017057,0.016742,0.017151,0.012269,0.010519,0.011107
min,0.233154,0.260010,0.245117,0.245117,0.002441,-0.009679,-0.010742
25%,0.264771,0.292236,0.276741,0.277771,0.018066,0.005418,0.006042
50%,0.280029,0.305542,0.291425,0.292114,0.024536,0.010046,0.010742
75%,0.289551,0.316467,0.302325,0.302490,0.032471,0.015758,0.017578
max,0.332520,0.346924,0.334659,0.335938,0.072144,0.058953,0.062744


### CLIP

In [76]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/ullx2qou")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,31.684453,37.589609,34.235234,34.545156,5.905156,2.550781,2.860703
std,4.032816,3.864454,3.573508,3.697707,3.136509,2.463453,2.567563
min,18.500000,28.000000,24.812500,24.578125,0.578125,-2.218750,-3.531250
25%,29.031250,34.640625,31.949219,31.917969,3.687500,0.960938,1.246094
50%,31.539062,37.718750,34.265625,34.437500,5.054688,2.007812,2.242188
75%,34.750000,40.343750,36.664062,37.218750,7.589844,3.714844,3.972656
max,42.593750,48.343750,44.593750,44.937500,17.093750,12.937500,13.218750


### JPEG

In [24]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/gz3b6q2k")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,115.629530,53.345975,64.290861,62.754610,-62.283555,-51.338670,-52.874920
std,27.508617,18.421629,20.472914,20.645551,24.887105,24.283941,24.494955
min,63.051998,15.890000,22.702116,18.319000,-182.047005,-174.057594,-175.105003
25%,96.998999,39.287500,50.387574,48.340750,-77.560498,-66.759896,-69.540252
50%,113.143501,51.469999,62.311882,61.207001,-59.809002,-48.798563,-50.538498
75%,131.945995,66.555748,77.573486,74.699749,-42.916752,-32.814957,-35.337749
max,236.546005,116.901001,153.927933,158.854004,-19.631996,-8.570526,-9.719994


## DNO

### Img Reward

In [103]:
data = get_wandb_logs(api, "pjajal/inference-dno/9vis1gzw")
compute_deltas_dnos(data, up_to_step=15).describe()

,baseline,pop_best,improvement
count,200.000000,200.000000,200.000000
mean,-0.082614,0.706908,0.789523
std,1.017341,0.925771,0.883648
min,-2.212891,-1.744141,-2.246582
25%,-0.765015,0.038269,0.183695
50%,-0.093689,0.910121,0.600830
75%,0.686523,1.512451,1.241638
max,1.845703,1.950374,3.990234


### HPSv2

In [78]:
data = get_wandb_logs(api, "pjajal/inference-dno/h26tfx2i")
compute_deltas_dnos(data, up_to_step=15).describe()

,baseline,pop_best,improvement
count,200.000000,200.000000,200.000000
mean,0.274603,0.286219,0.011616
std,0.016887,0.017411,0.012487
min,0.221313,0.240967,-0.038818
25%,0.262573,0.274109,0.004395
50%,0.275513,0.286255,0.010986
75%,0.288513,0.299805,0.018066
max,0.313477,0.324110,0.068604


### CLIP

In [34]:
data = get_wandb_logs(api, "pjajal/inference-dno/c3otmlnd")
compute_deltas_dnos(data, up_to_step=15).describe()

,baseline,pop_best,improvement
count,200.000000,200.000000,200.000000
mean,22.449453,26.236761,3.787308
std,2.389422,3.637785,2.997734
min,15.718750,18.046875,-0.421875
25%,21.101562,23.026242,1.222934
50%,22.234375,26.309099,2.988384
75%,23.476562,28.947326,5.796875
max,30.203125,35.562500,12.671875


### JPEG

In [23]:
data = get_wandb_logs(api, "pjajal/inference-dno/6enuouop")
compute_deltas_dnos(data, up_to_step=15, max_or_min="min").describe()

,pop_best,baseline,improvement
count,200.000000,0.0,0.0
mean,94.640115,NaN,NaN
std,22.776709,NaN,NaN
min,47.110001,NaN,NaN
25%,78.634750,NaN,NaN
50%,91.776501,NaN,NaN
75%,107.053249,NaN,NaN
max,174.445007,NaN,NaN


## SD Cosyne

### Img Reward

In [79]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/p5selqpd")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.035915,1.614149,1.370119,1.378829,1.578233,1.334203,1.342913
std,1.058178,0.417252,0.566536,0.573370,1.024946,0.997066,1.010158
min,-2.229451,-0.092948,-0.618320,-0.608302,0.084586,-0.014937,-0.021542
25%,-0.672078,1.489509,1.110918,1.132617,0.713431,0.504703,0.490776
50%,0.137491,1.765213,1.543249,1.564690,1.396616,1.082124,1.092388
75%,0.847085,1.884118,1.815400,1.818406,2.244661,1.938503,1.925829
max,1.876520,2.001776,1.992454,1.993146,4.092068,4.070545,4.072615


### HPSv2

In [80]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/4p9ka0yv")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.277531,0.310094,0.300125,0.299719,0.032563,0.022594,0.022189
std,0.018112,0.016525,0.016382,0.016421,0.013393,0.012190,0.012221
min,0.233154,0.270508,0.259460,0.259277,0.006592,-0.002472,-0.002686
25%,0.264771,0.299011,0.288067,0.287720,0.022705,0.012901,0.012634
50%,0.280029,0.311035,0.302002,0.301636,0.030518,0.021416,0.020752
75%,0.289551,0.321289,0.312466,0.312744,0.039673,0.027832,0.027588
max,0.332520,0.352051,0.340775,0.339600,0.072632,0.062363,0.062744


### CLIP

In [104]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/lxdmzzpo")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,31.684453,38.791328,36.529292,36.454922,7.106875,4.844839,4.770469
std,4.032816,3.811949,3.578963,3.613618,3.387622,3.081431,3.077962
min,18.500000,30.328125,28.550781,28.218750,1.218750,-0.107422,-0.156250
25%,29.031250,35.781250,34.312744,34.210938,4.585938,2.741699,2.652344
50%,31.539062,38.671875,36.093750,36.140625,6.109375,4.015625,3.843750
75%,34.750000,41.484375,39.172363,38.859375,9.093750,6.568359,6.542969
max,42.593750,50.062500,46.882812,46.937500,20.312500,16.935547,16.687500


### JPEG

In [26]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/5g3otp1e")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,115.629530,39.523165,45.027150,44.758985,-76.106365,-70.602380,-70.870545
std,27.508617,15.876781,16.276318,16.400938,27.784734,27.636138,27.599824
min,63.051998,13.779000,15.429188,15.423000,-199.767006,-197.204567,-196.811005
25%,96.998999,25.397000,30.748124,30.243249,-93.753250,-87.917467,-87.939251
50%,113.143501,36.788000,42.387501,42.414501,-75.110498,-71.139155,-71.369499
75%,131.945995,52.730500,58.911922,58.719252,-52.823501,-47.461876,-47.241499
max,236.546005,79.336998,85.922562,87.250999,-25.723999,-20.501316,-20.816002


### Rot

### Image Reward

In [93]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/sspqm9uy")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.035915,1.531776,1.225925,1.225250,1.495861,1.190009,1.189335
std,1.058178,0.436858,0.617000,0.633499,1.008532,0.950301,0.960091
min,-2.229451,-0.258774,-0.803441,-0.811372,0.054399,-0.329270,-0.335338
25%,-0.672078,1.367232,0.818715,0.845888,0.667455,0.413847,0.392734
50%,0.137491,1.658797,1.405625,1.419823,1.321334,0.936332,0.914392
75%,0.847085,1.857815,1.742325,1.738876,2.142321,1.828577,1.812024
max,1.876520,2.000602,1.975128,1.982139,4.030253,3.995003,3.987630


### HPSv2

In [10]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/ktgd80b8")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.277531,0.307235,0.298886,0.298627,0.029705,0.021355,0.021096
std,0.018112,0.017483,0.017127,0.017139,0.013198,0.012292,0.012347
min,0.233154,0.266846,0.261368,0.262451,0.002686,-0.004852,-0.006592
25%,0.264771,0.294189,0.285366,0.284912,0.020264,0.013424,0.013367
50%,0.280029,0.308350,0.300880,0.300049,0.028381,0.019745,0.019043
75%,0.289551,0.319824,0.311161,0.310974,0.036682,0.026871,0.026276
max,0.332520,0.346680,0.339584,0.339844,0.084961,0.070831,0.070801


### CLIP

In [12]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/9f6jrs07")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,31.684453,38.404688,36.453950,36.392812,6.720234,4.769497,4.708359
std,4.032816,3.795098,3.709481,3.712316,3.244789,2.961595,2.966362
min,18.500000,30.125000,26.163086,26.265625,1.015625,-0.120117,-0.109375
25%,29.031250,35.593750,34.076660,33.843750,4.492188,2.726074,2.578125
50%,31.539062,37.937500,36.226074,36.265625,5.773438,3.922852,3.867188
75%,34.750000,41.320312,39.057617,38.882812,8.757812,6.307617,6.484375
max,42.593750,50.187500,46.955078,47.250000,18.843750,16.751953,16.625000


### JPEG

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/xcva28u6")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,115.629530,38.729965,42.641697,42.276650,-76.899565,-72.987833,-73.352880
std,27.508617,16.253219,16.561559,16.634062,29.116999,29.204465,29.230375
min,63.051998,9.958000,12.285313,11.911000,-215.233006,-213.289192,-213.391005
25%,96.998999,27.349750,31.384420,30.684501,-96.494000,-93.126889,-93.501249
50%,113.143501,36.817501,41.625221,40.859499,-73.629501,-70.180936,-70.131500
75%,131.945995,46.703000,49.809937,49.383751,-54.807002,-51.573388,-51.939749
max,236.546005,108.240997,111.669495,111.333000,-21.424999,-18.318497,-18.198997


## SD SNES

### Img Reward

In [105]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/yxq7l3oo")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.035915,1.391709,0.704724,0.787659,1.355794,0.668808,0.751743
std,1.058178,0.523346,0.766267,0.800523,0.951857,0.812782,0.852293
min,-2.229451,-0.402447,-1.327335,-1.386546,0.087315,-0.730822,-0.668614
25%,-0.672078,1.108612,0.166353,0.192583,0.565178,0.110294,0.161316
50%,0.137491,1.534660,0.850373,0.933905,1.203921,0.505387,0.542231
75%,0.847085,1.812005,1.248090,1.469237,1.888701,1.011792,1.072933
max,1.876520,1.977036,1.927174,1.944097,4.049063,3.579776,3.885332


### HPSv2

In [106]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/l1461i7v")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.277531,0.300248,0.285404,0.286013,0.022717,0.007874,0.008482
std,0.018112,0.016065,0.015913,0.016141,0.011460,0.010033,0.010144
min,0.233154,0.259521,0.247888,0.247314,-0.000244,-0.011822,-0.010986
25%,0.264771,0.289307,0.273970,0.273865,0.014343,0.000351,0.001160
50%,0.280029,0.300781,0.287775,0.287964,0.020752,0.007025,0.007568
75%,0.289551,0.312012,0.297034,0.298096,0.028687,0.012567,0.013245
max,0.332520,0.345947,0.323072,0.324463,0.061279,0.041497,0.042969


### CLIP

In [107]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/j1yao8sr")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,31.684453,38.061328,35.372437,35.623438,6.376875,3.687984,3.938984
std,4.032816,3.750809,3.500092,3.563337,3.128541,2.660726,2.750112
min,18.500000,29.093750,24.603220,24.546875,1.843750,-2.321022,-2.250000
25%,29.031250,35.398438,33.160393,33.218750,4.148438,2.007577,2.171875
50%,31.539062,37.875000,35.132103,35.328125,5.492188,3.115059,3.210938
75%,34.750000,40.695312,37.414183,37.960938,8.097656,4.732482,4.972656
max,42.593750,49.562500,44.728222,44.968750,19.218750,16.803032,17.187500


### JPEG

In [35]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/pw8cxs22")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,115.629530,71.346320,78.328640,77.976620,-44.283210,-37.300890,-37.652910
std,27.508617,21.357172,22.351597,22.330866,18.283416,17.455129,17.407318
min,63.051998,28.472000,32.116123,32.361000,-154.501007,-147.757370,-147.988007
25%,96.998999,57.420752,63.377772,62.977251,-55.464252,-47.730879,-48.284998
50%,113.143501,67.539001,73.881184,73.695000,-41.960997,-34.970695,-35.519001
75%,131.945995,82.459249,91.155067,90.549002,-31.817251,-25.277374,-25.481752
max,236.546005,157.804001,168.810638,168.498993,-10.049995,0.112251,-0.985001


### Rot

### Image Reward

In [31]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/91rwif2q")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.035915,1.338316,0.857070,0.941777,1.302400,0.821154,0.905862
std,1.058178,0.560641,0.702255,0.726988,0.983300,0.880228,0.910371
min,-2.229451,-0.744166,-1.451384,-1.540792,-0.334090,-1.315905,-1.108929
25%,-0.672078,1.099203,0.448026,0.494828,0.477124,0.161048,0.200099
50%,0.137491,1.481660,1.001678,1.131704,1.105459,0.613711,0.682509
75%,0.847085,1.760126,1.379603,1.503761,1.936449,1.343412,1.440417
max,1.876520,1.982523,1.920120,1.954368,3.908481,3.186237,3.553808


### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/ll4ozqve")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.277531,0.300421,0.289414,0.290143,0.022891,0.011884,0.012613
std,0.018112,0.017417,0.016934,0.017080,0.012741,0.011853,0.012056
min,0.233154,0.250488,0.243065,0.243042,-0.001953,-0.015884,-0.014893
25%,0.264771,0.290710,0.277706,0.278259,0.013184,0.003300,0.003510
50%,0.280029,0.301758,0.292168,0.292969,0.020996,0.011021,0.011230
75%,0.289551,0.312805,0.301956,0.302673,0.031494,0.017811,0.018555
max,0.332520,0.341797,0.329376,0.333252,0.069092,0.061874,0.062500


### CLIP

In [9]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/1fdjdwx6")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,31.684453,37.406719,34.480156,34.699062,5.722266,2.795703,3.014609
std,4.032816,3.830964,3.588657,3.662344,3.101503,2.652113,2.782883
min,18.500000,29.250000,25.791016,26.593750,0.218750,-1.720703,-1.812500
25%,29.031250,34.437500,32.094971,32.265625,3.542969,1.216309,1.210938
50%,31.539062,37.187500,34.244141,34.437500,4.968750,2.157715,2.296875
75%,34.750000,40.007812,36.679199,37.054688,7.328125,4.104492,4.292969
max,42.593750,48.156250,44.859375,45.812500,19.593750,16.000000,16.562500


### JPEG

In [11]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/zmwqycd2")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,115.629530,57.783035,69.003270,66.096005,-57.846495,-46.626260,-49.533525
std,27.508617,22.396161,23.690381,23.359744,27.722469,28.542061,28.346948
min,63.051998,11.539000,18.264938,16.723000,-163.166008,-140.799690,-150.000008
25%,96.998999,42.582750,54.469642,51.217499,-75.461250,-65.654423,-68.841002
50%,113.143501,54.627501,65.773376,62.250000,-53.494499,-44.368345,-45.730501
75%,131.945995,70.697500,81.183405,78.210247,-37.347502,-26.570171,-29.100248
max,236.546005,174.643997,188.585312,185.453003,-0.056000,23.337807,22.378006


## SD PGPE

### Img Reward

In [30]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/w5sjg6wc")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.035915,1.261161,0.896539,0.921442,1.225246,0.860623,0.885526
std,1.058178,0.645635,0.796289,0.801340,0.884223,0.834261,0.846888
min,-2.229451,-1.166222,-1.640992,-1.681239,0.071018,-0.225553,-0.171246
25%,-0.672078,0.914310,0.350781,0.371503,0.527010,0.283817,0.291152
50%,0.137491,1.418985,1.069489,1.112125,1.046495,0.595365,0.621568
75%,0.847085,1.771437,1.541434,1.554925,1.666432,1.153547,1.202790
max,1.876520,1.977910,1.947841,1.951301,4.001263,3.936658,3.941486


### HPSv2

In [83]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/5c0hh9jn")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.277531,0.298867,0.289447,0.289587,0.021337,0.011916,0.012057
std,0.018112,0.017130,0.016872,0.016993,0.010459,0.009091,0.009167
min,0.233154,0.256348,0.252434,0.252197,0.003418,-0.004715,-0.004395
25%,0.264771,0.286804,0.277004,0.277039,0.013672,0.005665,0.005554
50%,0.280029,0.300903,0.290482,0.290527,0.019653,0.010849,0.010742
75%,0.289551,0.311340,0.301945,0.302124,0.027161,0.016739,0.017151
max,0.332520,0.341553,0.334412,0.335449,0.070435,0.048630,0.049683


### CLIP

In [90]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/jd8c8l67")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,31.684453,37.051875,34.317905,34.396719,5.367422,2.633452,2.712266
std,4.032816,3.607407,3.417293,3.455033,2.951372,2.428392,2.497550
min,18.500000,29.250000,26.083984,25.859375,0.406250,-3.134766,-3.765625
25%,29.031250,34.398438,32.047119,31.976562,3.386719,1.135742,1.160156
50%,31.539062,36.656250,34.075195,34.031250,4.570312,2.083984,2.179688
75%,34.750000,39.812500,36.391113,36.507812,6.683594,3.719238,3.777344
max,42.593750,47.843750,43.585938,43.468750,16.328125,14.525391,14.343750


### JPEG

In [27]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/1wo6mnfn")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,115.629530,88.292920,97.912356,97.018785,-27.336610,-17.717175,-18.610745
std,27.508617,23.683600,24.064785,24.017636,17.008179,16.130894,16.080008
min,63.051998,31.790001,54.369877,53.089001,-141.841003,-130.944824,-130.552002
25%,96.998999,72.262751,81.207376,79.728752,-34.227001,-24.181889,-25.312008
50%,113.143501,84.500000,93.367874,93.346500,-23.682499,-14.956127,-15.814995
75%,131.945995,103.504501,114.119030,112.593250,-16.336496,-6.927799,-8.240250
max,236.546005,160.106995,173.906494,173.785004,4.688004,12.665314,11.479004


### Rot

### Image Reward

In [32]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/heiropgw")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.035915,1.276422,0.845755,0.882238,1.240507,0.809839,0.846323
std,1.058178,0.577362,0.724125,0.731325,0.963229,0.913000,0.918535
min,-2.229451,-0.641304,-1.309703,-1.392634,0.013517,-0.915068,-0.767384
25%,-0.672078,0.938545,0.298907,0.332320,0.473873,0.094785,0.126877
50%,0.137491,1.425783,0.963430,1.003500,0.973225,0.546829,0.554418
75%,0.847085,1.737089,1.412063,1.448649,1.830407,1.384887,1.473696
max,1.876520,1.980671,1.968496,1.970089,3.961624,3.714071,3.765612


### CLIP

In [9]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/f4709ql9")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,31.684453,36.883906,34.687524,34.772031,5.199453,3.003071,3.087578
std,4.032816,3.763184,3.544234,3.601548,3.052783,2.730885,2.797946
min,18.500000,28.562500,27.024414,27.171875,-0.218750,-2.138672,-2.468750
25%,29.031250,33.859375,32.052246,31.972656,3.042969,1.126953,1.152344
50%,31.539062,36.609375,34.396973,34.515625,4.554688,2.387695,2.453125
75%,34.750000,39.226562,37.255859,37.476562,6.597656,4.346924,4.316406
max,42.593750,48.875000,44.912109,44.750000,18.531250,14.513672,15.687500


### HPSv2

In [33]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/8el737q9")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.277531,0.300153,0.290228,0.290478,0.022622,0.012697,0.012947
std,0.018112,0.017141,0.017103,0.017198,0.012404,0.011773,0.012025
min,0.233154,0.250977,0.244492,0.243042,-0.001221,-0.008301,-0.018311
25%,0.264771,0.288879,0.277401,0.277527,0.013184,0.004951,0.004608
50%,0.280029,0.301392,0.293350,0.293579,0.021240,0.010757,0.010986
75%,0.289551,0.313232,0.304367,0.304504,0.031067,0.020008,0.020020
max,0.332520,0.337646,0.328003,0.328857,0.067139,0.062515,0.062256


### JPEG

In [12]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/k3sgr4xu")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,115.629530,18.287230,20.484084,20.175150,-97.342300,-95.145446,-95.454380
std,27.508617,12.370529,13.552752,13.096625,30.499917,31.026680,30.930962
min,63.051998,5.003000,5.786500,5.380000,-227.806005,-226.472380,-226.456005
25%,96.998999,11.705000,12.946875,12.716000,-116.227251,-114.222564,-114.289751
50%,113.143501,14.756500,16.741687,16.597500,-93.877998,-91.015312,-92.745498
75%,131.945995,19.562749,21.806296,21.638501,-78.005749,-75.128439,-75.201751
max,236.546005,91.336998,95.110870,94.778000,-17.695999,-8.709000,-11.084999


# Open Image Preferences

## SD3

### Random

#### CLIP

In [11]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/pym27hnm")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.051823,38.995833,35.865625,36.006771,3.944010,0.813802,0.954948
std,3.813380,3.181348,3.117308,3.186333,2.101123,1.696725,1.672186
min,27.609375,31.156250,28.062500,28.125000,0.562500,-2.437500,-2.562500
25%,32.363281,36.851562,33.656250,33.648438,2.492188,-0.148438,0.062500
50%,34.531250,38.468750,35.843750,36.062500,3.539062,0.726562,0.875000
75%,37.476562,41.765625,38.328125,38.578125,5.273438,1.851562,1.910156
max,42.687500,45.093750,42.093750,42.000000,10.828125,4.484375,4.734375


#### ImageReward

In [10]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/ddwgfeu6")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.272554,1.719373,1.383061,1.441136,0.446819,0.110508,0.168583
std,0.529122,0.231283,0.383031,0.370158,0.406995,0.306122,0.325980
min,0.042161,0.810538,0.531865,0.573196,0.005380,-0.498201,-0.332433
25%,0.849486,1.609761,1.132849,1.230536,0.115242,-0.064370,-0.027535
50%,1.282937,1.773225,1.359116,1.459253,0.384602,0.044775,0.079164
75%,1.761907,1.878581,1.703432,1.735690,0.583598,0.222758,0.280099
max,1.977914,1.999845,1.969585,1.980922,1.809781,1.156780,1.466670


### Zero-Order

#### CLIP

In [12]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/cvxomgxz")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.051823,39.409896,37.234635,37.350521,4.358073,2.182812,2.298698
std,3.813380,3.505260,3.377296,3.403107,1.999701,1.640418,1.684587
min,27.609375,30.125000,28.359375,28.468750,1.718750,-0.500000,-0.625000
25%,32.363281,36.718750,34.359375,34.335938,2.726562,0.992188,1.195312
50%,34.531250,39.453125,37.171875,37.234375,3.750000,1.914062,2.023438
75%,37.476562,42.367188,39.914062,40.156250,5.550781,2.894531,3.019531
max,42.687500,46.031250,43.281250,43.343750,10.984375,6.265625,6.734375


#### ImageReward

In [13]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/2621vyt4")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.272554,1.752170,1.570616,1.603963,0.479617,0.298062,0.331409
std,0.529122,0.212686,0.321168,0.302778,0.408487,0.329598,0.346661
min,0.042161,0.963597,0.691993,0.713482,0.010524,-0.156684,-0.098262
25%,0.849486,1.674967,1.373835,1.425354,0.136921,0.044715,0.068443
50%,1.282937,1.797821,1.640217,1.674231,0.451346,0.182518,0.213601
75%,1.761907,1.901446,1.830567,1.851777,0.692117,0.492659,0.531747
max,1.977914,1.996979,1.986305,1.988518,1.768714,1.353175,1.479110


### CoSyne

#### CLIP

In [14]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/wg7e1kib")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.051823,40.631250,39.086475,39.032292,5.579427,4.034652,3.980469
std,3.813380,3.083478,3.073699,3.092768,2.155101,1.905187,1.912608
min,27.609375,33.875000,32.228516,32.093750,2.375000,0.992188,0.875000
25%,32.363281,38.414062,36.690430,36.648438,3.703125,2.346680,2.187500
50%,34.531250,40.656250,39.325195,39.140625,5.265625,3.829102,3.828125
75%,37.476562,43.085938,41.411621,41.421875,6.656250,5.250244,5.339844
max,42.687500,45.812500,44.414062,44.375000,10.609375,9.746094,9.734375


#### ImageReward

In [15]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/rwouqy0p")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.272554,1.820464,1.746185,1.746816,0.547910,0.473632,0.474262
std,0.529122,0.165190,0.217249,0.216668,0.437029,0.413623,0.413754
min,0.042161,1.199705,1.082086,1.082498,0.015523,0.011023,0.010523
25%,0.849486,1.746182,1.633858,1.640207,0.171784,0.158778,0.159056
50%,1.282937,1.863166,1.808311,1.810962,0.498423,0.403160,0.408541
75%,1.761907,1.928973,1.911877,1.912155,0.788529,0.695776,0.696930
max,1.977914,2.009863,2.003260,2.003989,1.847321,1.829627,1.830134


### SNES

#### CLIP

In [17]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/tp90h3fv")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.051823,39.886979,37.976521,38.106771,4.835156,2.924698,3.054948
std,3.813380,3.562881,3.471461,3.492797,1.942017,1.642234,1.727286
min,27.609375,30.656250,28.603464,28.546875,1.875000,-0.565876,-0.156250
25%,32.363281,37.414062,35.561234,35.726562,3.175781,1.524283,1.554688
50%,34.531250,39.625000,37.946793,38.078125,4.734375,3.030828,3.031250
75%,37.476562,43.046875,40.754858,40.914062,5.695312,3.813345,3.945312
max,42.687500,46.781250,44.352196,44.375000,10.734375,6.633869,7.796875


#### ImageReward

In [16]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/iq0vlq8v")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.272554,1.773404,1.661910,1.680269,0.500850,0.389357,0.407716
std,0.529122,0.216961,0.293086,0.280005,0.398328,0.354649,0.365918
min,0.042161,0.918722,0.780164,0.788713,0.018539,0.012428,0.013373
25%,0.849486,1.662966,1.493366,1.510324,0.157203,0.118169,0.125092
50%,1.282937,1.854126,1.757108,1.774256,0.455356,0.301261,0.323464
75%,1.761907,1.936742,1.885403,1.889255,0.679423,0.531834,0.571413
max,1.977914,2.001847,1.997784,1.998243,1.836477,1.709244,1.759732


### PGPE

#### CLIP

In [19]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/lovj86kc")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.051823,38.440625,36.431917,36.449740,3.388802,1.380094,1.397917
std,3.813380,3.636000,3.542673,3.566265,1.619994,1.090216,1.082889
min,27.609375,29.718750,28.204102,28.484375,0.843750,-1.484375,-1.375000
25%,32.363281,36.187500,34.057129,34.070312,1.992188,0.588867,0.593750
50%,34.531250,38.390625,36.137695,36.171875,3.359375,1.429688,1.421875
75%,37.476562,41.234375,38.964844,39.109375,4.339844,1.936523,1.945312
max,42.687500,45.656250,43.095703,43.031250,8.234375,4.579102,4.843750


#### ImageReward

In [18]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/3uvnm0gm")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.272554,1.673955,1.484772,1.501160,0.401401,0.212218,0.228606
std,0.529122,0.286684,0.393068,0.392958,0.355880,0.275762,0.288541
min,0.042161,0.692925,0.478233,0.513140,0.011363,-0.119875,-0.080811
25%,0.849486,1.501644,1.238612,1.242027,0.121985,0.029264,0.038024
50%,1.282937,1.779457,1.564176,1.589922,0.336094,0.120983,0.128514
75%,1.761907,1.880947,1.794548,1.809035,0.540311,0.309015,0.327310
max,1.977914,1.996155,1.984743,1.984841,1.757824,1.330769,1.459402


## PixArt

### Random

#### CLIP

In [9]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/huuj70g6")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.167187,38.761458,35.773698,35.910417,3.594271,0.606510,0.743229
std,2.962122,2.999448,2.922144,2.977670,1.793495,1.488473,1.470063
min,28.812500,31.062500,28.093750,28.000000,0.000000,-2.000000,-1.562500
25%,33.093750,36.476562,33.921875,34.062500,2.343750,-0.375000,-0.234375
50%,34.921875,39.031250,35.906250,36.031250,3.296875,0.601562,0.656250
75%,37.664062,41.023438,38.070312,38.437500,4.515625,1.335938,1.453125
max,41.156250,44.968750,42.812500,42.906250,8.937500,5.843750,6.156250


#### ImageReward

In [10]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/0q4jjz0r")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.108605,1.675975,1.193912,1.251725,0.567371,0.085307,0.143120
std,0.627217,0.255343,0.530678,0.536909,0.517647,0.309816,0.321732
min,-0.783014,0.660955,-0.519361,-0.564537,0.050391,-0.775585,-0.687749
25%,0.744646,1.595676,0.949973,1.066619,0.200138,-0.038881,-0.015195
50%,1.235360,1.724516,1.283446,1.363861,0.372685,0.065960,0.100129
75%,1.604742,1.848406,1.596388,1.632795,0.852354,0.171874,0.223671
max,1.944179,1.994570,1.973930,1.976211,2.176782,1.058426,1.399576


### Zero-Order

#### CLIP

In [11]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/cxkdtse9")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.211198,38.878906,36.012240,36.133333,3.667708,0.801042,0.922135
std,3.257001,3.116209,2.932207,2.991935,1.961989,1.677897,1.732539
min,27.859375,30.734375,27.734375,27.953125,0.000000,-3.687500,-3.562500
25%,32.773438,36.632812,34.109375,34.070312,2.281250,-0.265625,-0.156250
50%,35.078125,38.703125,35.812500,36.046875,3.460938,0.578125,0.640625
75%,37.492188,41.226562,38.257812,38.484375,4.796875,1.656250,1.695312
max,42.781250,45.062500,42.937500,43.125000,8.781250,5.593750,6.375000


#### ImageReward

In [12]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/ndlxn1i8")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.069886,1.672108,1.255947,1.291627,0.602222,0.186060,0.221741
std,0.651587,0.269232,0.510593,0.540149,0.515442,0.316619,0.310926
min,-0.656765,0.570324,-0.453540,-0.496727,-0.004259,-0.487553,-0.260100
25%,0.671747,1.548501,1.006374,1.051247,0.217136,-0.021235,-0.001734
50%,1.171896,1.722091,1.341250,1.433788,0.536620,0.109391,0.145436
75%,1.594629,1.850811,1.635502,1.692027,0.764622,0.394816,0.379294
max,1.966171,1.995664,1.971439,1.974956,2.344043,1.303886,1.342421


### CoSyne

#### CLIP

In [13]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/id3fc40g")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,34.989583,39.215625,36.906331,36.839062,4.226042,1.916748,1.849479
std,3.410699,3.016011,2.812615,2.811434,2.056033,1.552273,1.550563
min,26.031250,31.250000,28.627930,28.562500,0.843750,-1.085938,-1.125000
25%,32.515625,36.843750,34.934082,34.820312,2.687500,0.883789,0.820312
50%,34.562500,39.234375,37.032227,37.062500,3.796875,1.374023,1.265625
75%,37.546875,41.296875,38.815918,38.562500,5.726562,2.866211,2.765625
max,43.218750,45.781250,43.251953,43.187500,9.562500,7.388672,7.156250


#### ImageReward

In [14]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/en5e7do2")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.213668,1.694079,1.442368,1.450606,0.480412,0.228700,0.236938
std,0.629141,0.275608,0.428793,0.426568,0.471198,0.357809,0.380618
min,-0.811240,0.619017,-0.377958,-0.342205,0.013992,-0.287393,-0.329309
25%,0.920705,1.614091,1.290775,1.300501,0.149129,0.008791,0.020097
50%,1.357331,1.758878,1.512616,1.511065,0.271662,0.120483,0.125809
75%,1.687949,1.866509,1.726892,1.730931,0.794868,0.411976,0.405214
max,1.981211,1.998215,1.980580,1.980485,2.330063,1.582473,1.869428


### SNES

#### CLIP

In [15]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/4z67tfcf")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.393750,38.895052,35.642181,35.766667,3.501302,0.248431,0.372917
std,3.178005,3.149624,2.978043,2.999687,1.499515,1.194333,1.210056
min,26.781250,30.859375,27.513514,27.812500,0.562500,-3.711994,-3.593750
25%,33.210938,36.390625,33.926944,34.023438,2.460938,-0.461781,-0.382812
50%,35.046875,39.109375,35.463682,35.640625,3.296875,0.240290,0.390625
75%,37.718750,41.468750,37.770061,37.898438,4.398438,1.035052,1.140625
max,41.656250,44.781250,42.646961,42.687500,8.468750,2.309124,2.468750


#### ImageReward

In [16]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/2ip5kgez")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.044479,1.677438,1.181116,1.213219,0.632959,0.136637,0.168740
std,0.746375,0.268027,0.535532,0.552685,0.632314,0.376731,0.401616
min,-1.229270,0.633564,-0.551643,-0.573331,-0.010100,-0.364151,-0.373868
25%,0.693383,1.589025,0.947385,0.977163,0.207962,-0.083955,-0.036686
50%,1.264744,1.703615,1.232482,1.312212,0.362842,0.023982,0.035118
75%,1.590671,1.860673,1.584405,1.631867,0.970014,0.277230,0.349766
max,1.978800,1.995403,1.969699,1.972560,3.078528,1.608358,1.792055


### PGPE

#### CLIP

In [17]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/qp92xk2i")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,35.076563,38.654427,35.742334,35.822917,3.577865,0.665771,0.746354
std,3.190428,3.161801,2.993474,3.010450,1.846982,1.218769,1.190237
min,26.046875,30.859375,27.814453,28.250000,0.656250,-1.703125,-1.531250
25%,32.835938,36.312500,33.896973,34.046875,2.335938,-0.209473,-0.007812
50%,34.906250,38.500000,35.052246,35.171875,3.203125,0.532227,0.609375
75%,37.171875,41.234375,38.022217,38.125000,4.812500,1.438965,1.507812
max,43.468750,44.937500,42.726562,42.687500,9.609375,3.478516,3.687500


#### ImageReward

In [18]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/fvunpi6b")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,1.030142,1.645734,1.212602,1.232169,0.615592,0.182460,0.202027
std,0.682993,0.275356,0.547410,0.564224,0.513289,0.419083,0.454079
min,-0.846262,0.646965,-0.467930,-0.545471,0.013099,-1.405283,-1.757020
25%,0.665959,1.564286,0.947971,1.020644,0.237425,0.005488,0.011751
50%,1.175523,1.673714,1.326282,1.377883,0.436011,0.082980,0.124817
75%,1.513876,1.835066,1.620701,1.656662,0.840932,0.302926,0.344046
max,1.958398,1.995687,1.967188,1.972860,2.001145,1.263409,1.344379
